In [5]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy.interpolate import interp1d
from scipy.stats import stats


from biosppy.signals import tools as st
from biosppy.signals import eda
from biosppy.signals import ecg
from biosppy.signals import emg
from biosppy.signals import resp
from biosppy.signals import tools as st
from biosppy import plotting, utils

from scipy import signal

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [6]:
# General purpose function
# =============================
# Common statistics
def signal_stats(signal=None):
    """Compute various metrics describing the signal.
    Parameters
    ----------
    signal : array
        Input signal.
    Returns
    -------
    mean : float
        Mean of the signal.
    median : float
        Median of the signal.
    max : float
        Maximum signal amplitude.
    var : float
        Signal variance (unbiased).
    stddev : float
        Standard signal deviation (unbiased).
    absdev : float
        Absolute signal deviation.
    kurtosis : float
        Signal kurtosis (unbiased).
    skew : float
        Signal skewness (unbiased).
    """
    
    # check inputs
    if signal is None:
        raise TypeError("Please specify an input signal.")
    l= len(signal)
    
    # ensure numpy
    signal = np.array(signal)

    # mean
    mean = np.mean(signal)

    # median
    if l>1:
        median = np.median(signal)
    else:
        median = float('nan')
        
    # maximum amplitude
    maxAmp = signal.max()
    
    # minimum amplitude
    minAmp = signal.min()
    
    # range
    rangeAmp = signal.max()-signal.min()
    
    # variance
    sigma2 = signal.var(ddof=1)

    # standard deviation
    sigma = signal.std(ddof=1)

    # mean first order diviation
    if l>1:
        mfd = np.mean(np.diff(signal))
    else:
        mfd =0
        
    # max gradient
    if l>1:
        mg = np.diff(signal).max()
    else:
        mg = 0
        
    # absolute deviation
    if l>1:
        ad = np.sum(np.abs(signal - median))
    else:
        ad=0
        
    # kurtosis
    kurt = stats.kurtosis(signal, bias=False)

    # skweness
    skew = stats.skew(signal, bias=False)

    # output
    args = (mean, median, maxAmp, minAmp, rangeAmp, sigma2, sigma, mfd, mg, ad, kurt, skew)
    names = ('mean', 'median', 'max','min','vrange', 'var', 'stddev', 'avgder','maxgra','absdev', 'kurtosis',
             'skewness')

    return (args,names)

# Down samping
def downsampling(signal,sampling_rate,new_rate):
    r=sampling_rate/new_rate
    n_signal=signal[range(0,len(signal),r)]
    rate=sampling_rate/(r*1.0)
    return {'sig':n_signal,'rate':rate}

In [7]:
#ecg related function
def uni_hr(hr,hr_ts,ts):
    """resample and iterpolate to generate equal interval heart rate
    Require
    --------
    from scipy.interpolate import interp1d
    
    Parameter
    ---------
    hr : heart rate value
    hr_ts : heart rate time stamp
    ts : desired equal interval time stamp
    
    Returns
    -------
    hrn: equal interval heart rate
    """
    hrn=[None]*len(ts)
    hrn=np.array(hrn)
    hrn[ts >= hr_ts[-1]] = hr[-1]
    hrn[ts <= hr_ts[0]] = hr[0]
    last=np.nonzero(ts>=hr_ts[-1])
    le=list(last[0])[0]
    first=np.nonzero(ts<=hr_ts[0])
    fe=list(first[0])[-1]
    f = interp1d(hr_ts, hr)
    hrn[fe:le+1]=f(ts[range(fe,le+1)])
    hrn=[float(i) for i in hrn]
    return hrn

In [8]:
# load raw ECG signal
#sig_ecg = chunkdf['ecg']
def ecgfeature(sig_ecg):
    sampling_rate=1000
    ts, filtered, rpeaks, templates_ts, templates, hr_ts,hr = ecg.ecg(signal=sig_ecg, sampling_rate=1000,show=False)
    
    #ECG
    ecg_data,ecg_key=signal_stats(filtered)
    ecg_key=['ECG_raw_'+ i for i in ecg_key]
    #ecg frequency 
    freqs,power=st.power_spectrum(filtered, sampling_rate,decibel=False)
    band0,=st.band_power(freqs,power,(0,0.8),decibel=False).__getnewargs__()
    band1,=st.band_power(freqs,power,(0,0.2),decibel=False).__getnewargs__()
    band2,=st.band_power(freqs,power,(0.2,0.4),decibel=False).__getnewargs__()
    ecgfre_data=(band0,band1,band2)
    ecgfre_key=['ECG_fre_b0','ECG_fre_b1','ECG_fre_b2']
        
    #IBI 
    diffbeat=np.diff(hr_ts)
    diffbeat_data,diffbeat_key=signal_stats(diffbeat)
    diffbeat_key= ['ECG_IBI_'+ i for i in diffbeat_key]
    
    # heart rate
    hr_data,hr_key= signal_stats(hr)
    hr_key= ['ECG_HR_'+ i for i in hr_key]
    
    # heat rate fre
    hrn=uni_hr(hr,hr_ts,ts)
    hrn=np.array(hrn)
    freqs,power=st.power_spectrum(hrn, sampling_rate,decibel=False)
    #plt.plot(freqs,power)
    #plt.show()
    band0,=st.band_power(freqs,power,(0.03,0.4),decibel=False).__getnewargs__()
    band1,=st.band_power(freqs,power,(0.03,0.15),decibel=False).__getnewargs__()
    band2,=st.band_power(freqs,power,(0.15,0.4),decibel=False).__getnewargs__()
    hrfre_data=(band0,band1,band2)
    hrfre_key=['ECG_HR_b0','ECG_HR_b1','ECG_HR_b2']    
    
    ecg_feature= ecg_data + ecgfre_data + diffbeat_data + hr_data + hrfre_data
    ecg_key=  ecg_key+ ecgfre_key + diffbeat_key +hr_key + hrfre_key
    #return {'key':ecg_key,'feature':ecg_feature}
    return (ecg_key,ecg_feature)
    #ecgfeature(sig_ecg)


In [9]:
# load raw EDA signal
#sig_eda = chunkdf['eda']
#eda.eda(signal=signal, sampling_rate=sampling_rate,show=False)
def edafeature(sig_eda):
    # load raw EDA signal
    edad=downsampling(sig_eda,1000,40)
    signal=edad['sig']
    sampling_rate=edad['rate']

    signal = np.array(signal)

    # remove spikes
    signalp, _ = st.smoother(signal=signal,
                              kernel='median',
                              size=11)

    # filter signal
    filtered, _, _ = st.filter_signal(signal=signalp,
                                 ftype='butter',
                                 band='lowpass',
                                 order=4,
                                 frequency=5,
                                 sampling_rate=sampling_rate)

    #tonic
    tonic, _, _ = st.filter_signal(signal=filtered,
                                 ftype='butter',
                                 band='lowpass',
                                 order=4,
                                 frequency=0.03,
                                 sampling_rate=sampling_rate)
    #phasic
    phasic = filtered - tonic


    df = np.diff(phasic)
    length=len(signal)

    T = (length - 1) / sampling_rate
    ts = np.linspace(0, T, length, endpoint=False)


    zeros, = st.zero_cross(signal=df, detrend=False)

    if np.all(df[:zeros[0]] > 0):
        zeros = zeros[1:]
    if np.all(df[zeros[-1]:] > 0):
        zeros = zeros[:-1]

    thr= 0.2 * np.max(df)

    scrs, amps, ZC, pks = [], [], [], []
    for i in range(0, len(zeros) - 1, 2):
        scrs += [df[zeros[i]:zeros[i + 1]]]
        aux1 = scrs[-1].max()
        if aux1 > thr:
            amps += [filtered[zeros[i+1]]]
            ZC += [zeros[i]]
            ZC += [zeros[i + 1]]
            pks += [zeros[i+1]] #+ np.argmax(df[zeros[i]:zeros[i + 1]])]

    scrs = np.array(scrs)
    amps = np.array(amps)
    ZC = np.array(ZC)
    pks = np.array(pks)
    onsets = ZC[::2]

    #     # get SCR info
    #     #onsets, peaks, amps = basic_scr(signal=df, sampling_rate=sampling_rate)
    #     scr_data,scr_key= signal_stats(amps)
    #     scr_key= ['scramps_'+ i for i in scr_key]

    scr_count=(len(onsets),)
    scrct_key=['EDA_one_nbpks',]

    edasig_data,edasig_key= signal_stats(filtered)
    edasig_key= ['EDA_raw_'+ i for i in edasig_key]

    #eda frequency 
    sig,=st.normalize(filtered)
    freqs,power=st.power_spectrum(sig, sampling_rate,decibel=False)
    band0,=st.band_power(freqs,power,(0,0.8),decibel=False).__getnewargs__()
    band1,=st.band_power(freqs,power,(0,0.2),decibel=False).__getnewargs__()
    band2,=st.band_power(freqs,power,(0.2,0.4),decibel=False).__getnewargs__()
    band3,=st.band_power(freqs,power,(0.4,0.6),decibel=False).__getnewargs__()
    band4,=st.band_power(freqs,power,(0.6,0.8),decibel=False).__getnewargs__()
    edafre_data=(band0,band1,band2,band3,band4)
    edafre_key=['EDA_fre_b0','EDA_fre_b1','EDA_fre_b2','EDA_fre_b3','EDA_fre_b4']
    
    eda_ph_data, eda_ph_key=signal_stats(phasic)
    eda_ph_key= ['EDA_ph_'+ i for i in eda_ph_key]

    eda_ph_de_data, eda_ph_de_key=signal_stats(np.diff(phasic))
    eda_ph_de_key= ['EDA_phde_'+ i for i in eda_ph_de_key]
    
    eda_to_data, eda_to_key=signal_stats(tonic)
    eda_to_key= ['EDA_tonic_'+ i for i in eda_to_key]

    eda_feature = edasig_data + edafre_data + eda_ph_data + eda_ph_de_data + eda_to_data+ scr_count
    eda_key =  edasig_key + edafre_key + eda_ph_key + eda_ph_de_key + eda_to_key + scrct_key 
    return eda_key,eda_feature

In [10]:
def emgfeature(sig_emg):
    emgd=downsampling(sig_emg,1000,40)
    signal=emgd['sig']
    sampling_rate=emgd['rate']
    #remove spike
    signalp, _ = st.smoother(signal=signal,
                                  kernel='median',
                                  size=11)
    #filter RMS
    signal=np.array(np.square(signalp))
    filtered, _ = st.smoother(signal=signal,
                                  kernel='boxzen',
                                  size=40,
                                  mirror=True)
    filtered,=st.normalize(filtered)
    thr=(max(filtered)-np.mean(filtered))*0.2 + np.mean(filtered)
    emgct_key= ['EMG_one_ct',]
    emgct_data= (sum(i > thr for i in filtered)/(len(filtered)*1.0),)

    #emg
    emgsig_data,emg_key= signal_stats(filtered)
    emgsig_key= ['EMG_raw_'+ i for i in emg_key]
    
    emg_key= emgsig_key + emgct_key
    emg_feature=emgsig_data +emgct_data
    
    return (emg_key,emg_feature)

In [11]:
def sktfeature(sig_skt):
    sktd=downsampling(sig_skt,1000,40)
    signal=sktd['sig']
    sampling_rate=sktd['rate']
    #remove spike
    filtered, _ = st.smoother(signal=signal,
                                  kernel='median',
                                  size=11)
    
#     filtered, _, _ = st.filter_signal(signal=filtered,
#                                       ftype='butter',
#                                       band='bandpass',
#                                       order=2,
#                                       frequency=[0.1, 0.35],
#                                       sampling_rate=sampling_rate)
    #skt
    sktsig_data,skt_key= signal_stats(filtered)
    sktsig_key= ['SKT_raw_'+ i for i in skt_key]
    
    skt_key= sktsig_key 
    skt_feature=sktsig_data 
    
    return (skt_key,skt_feature)

In [12]:
def rspfeature(sig_resp):
    respd=downsampling(sig_resp,1000,40)
    msignal=respd['sig']
    sampling_rate=respd['rate']
    
    sig_resp, _ = st.smoother(signal=msignal,
                              kernel='median',
                              size=11)

    sig_resps, _, _ = st.filter_signal(signal=signal.detrend(sig_resp, type='constant'),
                                     ftype='butter',
                                     band='lowpass',
                                     order=4,
                                     frequency=3,
                                     sampling_rate=sampling_rate)
    sig_resps=np.convolve(sig_resps, np.ones((sampling_rate,))/sampling_rate, mode='same')
    N=sampling_rate*3
    sig_respde=sig_resps-np.convolve(sig_resps, np.ones((N,))/N, mode='same')
    # compute zero crossings

    sig_respde=signal.detrend(sig_respde)
    zerocrossing=np.where(sig_respde[:-1]*sig_respde[1:]<0)
    beats = zerocrossing[0][::2]

    rspn=len(beats)

    resp_rate=sampling_rate * (1. / np.diff(beats))

    respdiff=np.diff(sig_respde)
    respdiffs=np.convolve(respdiff, np.ones((10,))/10, mode='same')
    resppeak= np.where(respdiffs[:-1]*respdiffs[1:]<0) 


    if (sig_respde[resppeak[0][:-1:2]]>0).sum()>0:
        uppeaks = resppeak[0][:-1:2]
        #downpeaks =  resppeak[0][1::2]
    else:
        uppeaks = resppeak[0][1::2]
        #downpeaks = resppeak[0][:-1:2]

    uppeaks=uppeaks[sig_respde[uppeaks]>0]   
    ampup = sig_respde[uppeaks]
    
    # statistics on respiration
    rspsig_data,rspsig_key = signal_stats(sig_respde)
    rspsig_key= ['Resp_raw_'+ i for i in rspsig_key]
    
    # resp frequency 
    freqs,power=st.power_spectrum(sig_respde, sampling_rate,decibel=False)
    band0,=st.band_power(freqs,power,(0,0.8),decibel=False).__getnewargs__()
    band1,=st.band_power(freqs,power,(0,0.2),decibel=False).__getnewargs__()
    band2,=st.band_power(freqs,power,(0.2,0.4),decibel=False).__getnewargs__()
    band3,=st.band_power(freqs,power,(0.4,0.6),decibel=False).__getnewargs__()
    band4,=st.band_power(freqs,power,(0.6,0.8),decibel=False).__getnewargs__()
    rspfre_data=(band0,band1,band2,band3,band4)
    rspfre_key=['Resp_fre_b0','Resp_fre_b1','Resp_fre_b2','Resp_fre_b3','Resp_fre_b4']    
    
    rspnum_data= (rspn,)
    rspnum_key= ['Resp_one_Num',]
    
    #statistics on respiration rate
    rsprt_data,rsprt_key= signal_stats(resp_rate)
    rsprt_key= ['Resp_RR_'+ i for i in rsprt_key]
    
    # amplitude
    resampup_data,resampup_key=signal_stats(ampup)
    resampup_key=['Resp_Ampup_'+ i for i in resampup_key]
    
    #resampdown_data,resampdown_key=signal_stats(ampdown)
    #resampdown_key=['ResAmpdown_'+ i for i in resampdown_key]

    resp_feature = rspsig_data + rspfre_data + rsprt_data + rspnum_data + resampup_data# + resampdown_data
    resp_key = rspsig_key + rspfre_key + rsprt_key + rspnum_key +resampup_key #+ resampdown_key

    #return {'key':resp_key,'feature':resp_feature}
    return resp_key, resp_feature

# Event feature

In [13]:
def allfeature(chunkdf):
    ecg_key, ecg_feature=ecgfeature(chunkdf['ecg'])
    emgf_key, emgf_feature=emgfeature(chunkdf['emg_f'])
    emgf_key=['f'+i for i in emgf_key]
    emgs_key, emgs_feature=emgfeature(chunkdf['emg_s'])
    emgs_key=['s'+i for i in emgf_key]
    eda_key, eda_feature=edafeature(chunkdf['eda'])
    rsp_key, rsp_feature=rspfeature(chunkdf['rsp'])

    key = ecg_key + emgf_key + emgs_key + eda_key + rsp_key
    feature= ecg_feature + emgf_feature + emgs_feature + eda_feature + rsp_feature
    return pd.DataFrame([list(feature)], columns=key)

In [14]:
filebase='/home/wenlu/Documents/XPFIFA/biopacEvent20s/S_58_1747_.txt' #basefilepath+fn
chunkdf=pd.read_csv(filebase,sep='\t',nrows=20000,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
   

In [15]:
tmp=allfeature(chunkdf)

/local/wenlu/anaconda2/lib/python2.7/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


In [16]:
tmp

,ECG_raw_mean,ECG_raw_median,ECG_raw_max,ECG_raw_min,ECG_raw_vrange,ECG_raw_var,ECG_raw_stddev,ECG_raw_avgder,ECG_raw_maxgra,ECG_raw_absdev,ECG_raw_kurtosis,ECG_raw_skewness,ECG_fre_b0,ECG_fre_b1,ECG_fre_b2,ECG_IBI_mean,ECG_IBI_median,ECG_IBI_max,ECG_IBI_min,ECG_IBI_vrange,ECG_IBI_var,ECG_IBI_stddev,ECG_IBI_avgder,ECG_IBI_maxgra,ECG_IBI_absdev,ECG_IBI_kurtosis,ECG_IBI_skewness,ECG_HR_mean,ECG_HR_median,ECG_HR_max,ECG_HR_min,ECG_HR_vrange,ECG_HR_var,ECG_HR_stddev,ECG_HR_avgder,ECG_HR_maxgra,ECG_HR_absdev,ECG_HR_kurtosis,ECG_HR_skewness,ECG_HR_b0,ECG_HR_b1,ECG_HR_b2,fEMG_raw_mean,fEMG_raw_median,fEMG_raw_max,fEMG_raw_min,fEMG_raw_vrange,fEMG_raw_var,fEMG_raw_stddev,fEMG_raw_avgder,fEMG_raw_maxgra,fEMG_raw_absdev,fEMG_raw_kurtosis,fEMG_raw_skewness,fEMG_one_ct,sfEMG_raw_mean,sfEMG_raw_median,sfEMG_raw_max,sfEMG_raw_min,sfEMG_raw_vrange,sfEMG_raw_var,sfEMG_raw_stddev,sfEMG_raw_avgder,sfEMG_raw_maxgra,sfEMG_raw_absdev,sfEMG_raw_kurtosis,sfEMG_raw_skewness,sfEMG_one_ct,EDA_raw_mean,EDA_raw_median,EDA_raw_max,EDA_raw_min,EDA_raw_vrange,EDA_raw_var,EDA_raw_stddev,EDA_raw_avgder,EDA_raw_maxgra,EDA_raw_absdev,EDA_raw_kurtosis,EDA_raw_skewness,EDA_fre_b0,EDA_fre_b1,EDA_fre_b2,EDA_fre_b3,EDA_fre_b4,EDA_ph_mean,EDA_ph_median,EDA_ph_max,EDA_ph_min,EDA_ph_vrange,EDA_ph_var,EDA_ph_stddev,EDA_ph_avgder,EDA_ph_maxgra,EDA_ph_absdev,EDA_ph_kurtosis,EDA_ph_skewness,EDA_phde_mean,EDA_phde_median,EDA_phde_max,EDA_phde_min,EDA_phde_vrange,EDA_phde_var,EDA_phde_stddev,EDA_phde_avgder,EDA_phde_maxgra,EDA_phde_absdev,EDA_phde_kurtosis,EDA_phde_skewness,EDA_tonic_mean,EDA_tonic_median,EDA_tonic_max,EDA_tonic_min,EDA_tonic_vrange,EDA_tonic_var,EDA_tonic_stddev,EDA_tonic_avgder,EDA_tonic_maxgra,EDA_tonic_absdev,EDA_tonic_kurtosis,EDA_tonic_skewness,EDA_one_nbpks,Resp_raw_mean,Resp_raw_median,Resp_raw_max,Resp_raw_min,Resp_raw_vrange,Resp_raw_var,Resp_raw_stddev,Resp_raw_avgder,Resp_raw_maxgra,Resp_raw_absdev,Resp_raw_kurtosis,Resp_raw_skewness,Resp_fre_b0,Resp_fre_b1,Resp_fre_b2,Resp_fre_b3,Resp_fre_b4,Resp_RR_mean,Resp_RR_median,Resp_RR_max,Resp_RR_min,Resp_RR_vrange,Resp_RR_var,Resp_RR_stddev,Resp_RR_avgder,Resp_RR_maxgra,Resp_RR_absdev,Resp_RR_kurtosis,Resp_RR_skewness,Resp_one_Num,Resp_Ampup_mean,Resp_Ampup_median,Resp_Ampup_max,Resp_Ampup_min,Resp_Ampup_vrange,Resp_Ampup_var,Resp_Ampup_stddev,Resp_Ampup_avgder,Resp_Ampup_maxgra,Resp_Ampup_absdev,Resp_Ampup_kurtosis,Resp_Ampup_skewness
0,0.000383,-0.003139,1.34215,-0.435423,1.777574,0.043001,0.207367,2.728175e-07,0.08795,1964.579949,17.589658,3.411546,5.249996e-07,4.381891e-07,4.907494e-07,0.831387,0.839958,0.924954,0.743963,0.180991,0.002883,0.053698,-0.0022,0.074996,0.957952,-1.118802,-0.050601,72.613487,72.50757,79.225383,66.616993,12.60839,16.582308,4.072138,0.172556,3.290536,76.104661,-1.322062,0.125479,4.406809,14.206179,0.487061,-1.065814e-16,0.169381,2.085082,-2.228703,4.313785,1.0,1.0,0.000574,0.141559,660.185291,-0.732529,-0.081123,0.34,-5.240253e-16,-0.060783,2.498613,-4.089151,6.587764,1.0,1.0,-0.001323,0.107913,606.001852,1.784799,-0.362875,0.31375,10.258991,10.065957,11.10107,9.88106,1.22001,0.138321,0.371915,0.000521,0.019772,256.451025,-0.692357,0.73502,0.124309,0.474149,0.019653,0.002619,0.000816,0.203478,-0.000316,1.026103,-0.180139,1.206242,0.125991,0.354953,0.000422,0.019698,239.979308,-0.537385,0.789072,0.000422,-0.000199,0.019698,-0.009284,0.028982,0.000023,0.004775,-4.187345e-07,0.001775,2.214151,4.606962,1.859118,10.055513,10.066272,10.078793,9.999305,0.079488,0.000633,0.025155,0.000099,0.000189,16.471717,-0.765572,-0.797512,1,-6.938894e-19,-0.01421,0.125778,-0.093926,0.219704,0.003382,0.058156,0.000071,0.00777,39.329734,-0.891107,0.422244,0.000418,0.000006,0.000357,0.001265,0.000042,0.38026,0.377358,0.421053,0.344828,0.076225,0.000613,0.024764,0.002589,0.032703,0.126422,0.270414,0.30243,8,0.095171,0.082962,0.125669,0.058225,0.067443,0.000721,0.026852,0.008024,0.033576,0.151238,-1.899212,0.04427


In [140]:
tmp.to_csv('./../out/feature/header.csv',encoding='utf-16',index=False)

In [64]:
eventfilepath='./../biopacBase20_random/'
phyEventdf=pd.DataFrame()
for fn in os.listdir(eventfilepath): 
    print(fn)
    filebase=eventfilepath+fn
    sujet=int(fn.split('_')[1])
    baseidx=int(re.split('_|\.',fn)[4])
    chunkdf=pd.read_csv(filebase,sep='\t',nrows=20000,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
    try:
        currentfeature=allfeature(chunkdf)
    except:
        print "feature calculation error\n"
        continue
    currentfeature['sujet']=sujet
    currentfeature['eventidx']=baseidx
    phyEventdf=phyEventdf.append(currentfeature)
    

S_8_245_.txt


ValueError: invalid literal for int() with base 10: 'txt'

In [50]:
phyEventdf.shape

(1730, 166)

In [51]:
phyEventdf=phyEventdf.sort(['sujet','eventidx'],ascending=[1,1])

In [ ]:
phyEventdf.drop('eventidx').head()

In [60]:
phyEventdf=pd.concat([phyEventdf['sujet'],phyEventdf.iloc[:,:-2]],axis=1)

In [61]:
phyEventdf.to_csv('./../out/feature/phyBaseRandom.csv',encoding='utf-16',index=False)

In [62]:
phyEventdf=pd.read_csv('./../out/feature/phyBaseRandom.csv',encoding='utf-16')

In [63]:
phyEventdf.head()

,sujet,ECG_mean,ECG_median,ECG_max,ECG_min,ECG_vrange,ECG_var,ECG_std_dev,ECG_avg_der,ECG_max_gra,ECG_abs_dev,ECG_kurtosis,ECG_skewness,ECG_b0,ECG_b1,ECG_b2,IBI_mean,IBI_median,IBI_max,IBI_min,IBI_vrange,IBI_var,IBI_std_dev,IBI_avg_der,IBI_max_gra,IBI_abs_dev,IBI_kurtosis,IBI_skewness,HR_mean,HR_median,HR_max,HR_min,HR_vrange,HR_var,HR_std_dev,HR_avg_der,HR_max_gra,HR_abs_dev,HR_kurtosis,HR_skewness,HR_b0,HR_b1,HR_b2,EMG_mean_f,EMG_median_f,EMG_max_f,EMG_min_f,EMG_vrange_f,EMG_var_f,EMG_std_dev_f,EMG_avg_der_f,EMG_max_gra_f,EMG_abs_dev_f,EMG_kurtosis_f,EMG_skewness_f,emg_ct_f,EMG_mean_f_s,EMG_median_f_s,EMG_max_f_s,EMG_min_f_s,EMG_vrange_f_s,EMG_var_f_s,EMG_std_dev_f_s,EMG_avg_der_f_s,EMG_max_gra_f_s,EMG_abs_dev_f_s,EMG_kurtosis_f_s,EMG_skewness_f_s,emg_ct_f_s,EDA_mean,EDA_median,EDA_max,EDA_min,EDA_vrange,EDA_var,EDA_std_dev,EDA_avg_der,EDA_max_gra,EDA_abs_dev,EDA_kurtosis,EDA_skewness,EDA_b0,EDA_b1,EDA_b2,EDA_b3,EDA_b4,EDA_ph_mean,EDA_ph_median,EDA_ph_max,EDA_ph_min,EDA_ph_vrange,EDA_ph_var,EDA_ph_std_dev,EDA_ph_avg_der,EDA_ph_max_gra,EDA_ph_abs_dev,EDA_ph_kurtosis,EDA_ph_skewness,EDA_ph_de_mean,EDA_ph_de_median,EDA_ph_de_max,EDA_ph_de_min,EDA_ph_de_vrange,EDA_ph_de_var,EDA_ph_de_std_dev,EDA_ph_de_avg_der,EDA_ph_de_max_gra,EDA_ph_de_abs_dev,EDA_ph_de_kurtosis,EDA_ph_de_skewness,EDA_to_mean,EDA_to_median,EDA_to_max,EDA_to_min,EDA_to_vrange,EDA_to_var,EDA_to_std_dev,EDA_to_avg_der,EDA_to_max_gra,EDA_to_abs_dev,EDA_to_kurtosis,EDA_to_skewness,EDA_nbpks,rsp_mean,rsp_median,rsp_max,rsp_min,rsp_vrange,rsp_var,rsp_std_dev,rsp_avg_der,rsp_max_gra,rsp_abs_dev,rsp_kurtosis,rsp_skewness,Resp_b0,Resp_b1,Resp_b2,Resp_b3,Resp_b4,RR_mean,RR_median,RR_max,RR_min,RR_vrange,RR_var,RR_std_dev,RR_avg_der,RR_max_gra,RR_abs_dev,RR_kurtosis,RR_skewness,ResNum,ResAmpup_mean,ResAmpup_median,ResAmpup_max,ResAmpup_min,ResAmpup_vrange,ResAmpup_var,ResAmpup_std_dev,ResAmpup_avg_der,ResAmpup_max_gra,ResAmpup_abs_dev,ResAmpup_kurtosis,ResAmpup_skewness
0,2,-0.000066,-0.001899,1.534833,-0.464670,1.999503,0.067692,0.260177,0.000000,0.104341,2303.823143,15.616583,3.352876,8.518451e-08,2.080879e-08,1.388915e-07,0.690850,0.685466,0.759962,0.638968,0.120994,0.001040,0.032254,-0.000400,0.045998,0.674966,-0.454893,0.382033,87.089248,87.182832,92.460984,80.233939,12.227046,11.798926,3.434957,0.121881,2.930110,74.612632,-0.748028,-0.345655,3.195052,9.685806,0.598751,7.105427e-17,-0.378269,3.777282,-1.553450,5.330732,1,1,0.001209,0.216363,547.023392,3.016066,1.670248,0.20500,4.618528e-16,0.078744,2.081176,-3.724736,5.805912,1,1,0.000641,0.263928,613.154773,0.801030,-0.770586,0.37500,10.518072,10.489506,10.953297,10.253419,0.699878,0.041582,0.203916,-0.000357,0.005602,139.173059,-0.811667,0.524069,0.124030,0.480681,0.013331,0.000930,0.001180,-0.100846,-0.131450,0.333764,-0.365529,0.699293,0.041692,0.204187,-0.000370,0.005547,139.611498,-0.826167,0.520438,-0.000370,-0.000173,0.005547,-0.005429,0.010976,0.000004,0.002052,-0.000003,0.001050,1.171118,0.658813,0.504569,10.618918,10.619339,10.621581,10.608237,0.013344,0.000008,0.002810,0.000013,0.000095,1.392266,3.616198,-1.971837,4,1.110223e-18,-0.025412,0.691969,-0.800399,1.492368,0.122754,0.350362,0.000135,0.069991,242.592023,-1.021760,0.006819,0.015265,0.001203,0.006729,0.049145,0.003982,0.403056,0.392157,0.506329,0.360360,0.145969,0.002550,0.050500,-0.021495,0.056306,0.233080,3.273273,1.685192,8,0.515705,0.511994,0.691969,0.338890,0.353080,0.014490,0.120375,-0.029996,0.173104,0.665935,-0.739475,0.000358
1,2,0.001032,-0.002285,1.604667,-0.468799,2.073467,0.069647,0.263907,-0.000001,0.107080,2340.890349,15.232376,3.334829,4.267959e-06,3.348276e-06,4.435655e-06,0.669596,0.668967,0.717964,0.638968,0.078996,0.000503,0.022439,0.000731,0.045998,0.498975,-0.653422,0.380257,89.491262,89.775209,93.604200,84.963770,8.640431,6.831768,2.613765,0.108422,2.930110,63.160717,-1.237586,-0.157133,1.842284,5.948354,0.199856,-1.953993e-16,-0.127666,2.768897,-1.812619,4.581516,1,1,0.001877,0.165524,627.890433,0.020734,0.682935,0.25125,-8.5

In [150]:
eventfilepath='./../biopacBeforeEvent14s/'
nbrows=14000
phyEventdf=pd.DataFrame()
for fn in os.listdir(eventfilepath): 
    print(fn)
    filebase=eventfilepath+fn
    sujet=int(fn.split('_')[1])
    eventidx=int(fn.split('_')[2])
    chunkdf=pd.read_csv(filebase,sep='\t',nrows=nbrows,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
    try:
        currentfeature=allfeature(chunkdf)
    except:
        print "feature calculation error\n"
        continue
    currentfeature['sujet']=sujet
    currentfeature['eventidx']=eventidx
    phyEventdf=phyEventdf.append(currentfeature)
    

S_8_245_.txt
S_50_1519_.txt
S_6_194_.txt
S_24_782_.txt
S_26_849_.txt
S_19_555_.txt
S_42_1301_.txt
S_5_179_.txt
S_52_1547_.txt
S_55_1655_.txt
S_12_341_.txt
S_18_522_.txt
S_38_1219_.txt
S_46_1403_.txt
S_19_554_.txt
S_40_1237_.txt
S_46_1412_.txt
S_57_1715_.txt
S_51_1527_.txt
S_43_1336_.txt
S_26_843_.txt
S_13_385_.txt
S_32_1036_.txt
S_7_207_.txt
S_10_294_.txt
S_47_1437_.txt
S_19_586_.txt
S_23_748_.txt
S_20_622_.txt
S_26_856_.txt
S_5_172_.txt
S_43_1319_.txt
S_36_1181_.txt
S_16_454_.txt
S_24_774_.txt
S_13_382_.txt
S_34_1077_.txt
S_10_292_.txt
S_56_1698_.txt
S_19_567_.txt
S_26_866_.txt
S_19_551_.txt
S_13_386_.txt
S_45_1398_.txt
S_10_301_.txt
S_12_343_.txt
S_36_1161_.txt
S_55_1678_.txt
S_14_413_.txt
S_24_778_.txt
S_48_1466_.txt
S_8_233_.txt
S_26_838_.txt
S_34_1104_.txt
S_15_431_.txt
S_40_1251_.txt
S_20_638_.txt
S_33_1046_.txt
S_15_430_.txt
S_16_459_.txt
S_45_1384_.txt
S_39_1220_.txt
S_5_162_.txt
S_41_1270_.txt
S_44_1352_.txt
S_17_488_.txt
S_54_1631_.txt
S_45_1392_.txt
S_35_1143_.txt
S_42_1300_

In [148]:
phyEventdf.head()

,ECG_raw_mean,ECG_raw_median,ECG_raw_max,ECG_raw_min,ECG_raw_vrange,ECG_raw_var,ECG_raw_stddev,ECG_raw_avgder,ECG_raw_maxgra,ECG_raw_absdev,ECG_raw_kurtosis,ECG_raw_skewness,ECG_fre_b0,ECG_fre_b1,ECG_fre_b2,ECG_IBI_mean,ECG_IBI_median,ECG_IBI_max,ECG_IBI_min,ECG_IBI_vrange,ECG_IBI_var,ECG_IBI_stddev,ECG_IBI_avgder,ECG_IBI_maxgra,ECG_IBI_absdev,ECG_IBI_kurtosis,ECG_IBI_skewness,ECG_HR_mean,ECG_HR_median,ECG_HR_max,ECG_HR_min,ECG_HR_vrange,ECG_HR_var,ECG_HR_stddev,ECG_HR_avgder,ECG_HR_maxgra,ECG_HR_absdev,ECG_HR_kurtosis,ECG_HR_skewness,ECG_HR_b0,ECG_HR_b1,ECG_HR_b2,fEMG_raw_mean,fEMG_raw_median,fEMG_raw_max,fEMG_raw_min,fEMG_raw_vrange,fEMG_raw_var,fEMG_raw_stddev,fEMG_raw_avgder,fEMG_raw_maxgra,fEMG_raw_absdev,fEMG_raw_kurtosis,fEMG_raw_skewness,fEMG_one_ct,sfEMG_raw_mean,sfEMG_raw_median,sfEMG_raw_max,sfEMG_raw_min,sfEMG_raw_vrange,sfEMG_raw_var,sfEMG_raw_stddev,sfEMG_raw_avgder,sfEMG_raw_maxgra,sfEMG_raw_absdev,sfEMG_raw_kurtosis,sfEMG_raw_skewness,sfEMG_one_ct,EDA_raw_mean,EDA_raw_median,EDA_raw_max,EDA_raw_min,EDA_raw_vrange,EDA_raw_var,EDA_raw_stddev,EDA_raw_avgder,EDA_raw_maxgra,EDA_raw_absdev,EDA_raw_kurtosis,EDA_raw_skewness,EDA_fre_b0,EDA_fre_b1,EDA_fre_b2,EDA_fre_b3,EDA_fre_b4,EDA_ph_mean,EDA_ph_median,EDA_ph_max,EDA_ph_min,EDA_ph_vrange,EDA_ph_var,EDA_ph_stddev,EDA_ph_avgder,EDA_ph_maxgra,EDA_ph_absdev,EDA_ph_kurtosis,EDA_ph_skewness,EDA_phde_mean,EDA_phde_median,EDA_phde_max,EDA_phde_min,EDA_phde_vrange,EDA_phde_var,EDA_phde_stddev,EDA_phde_avgder,EDA_phde_maxgra,EDA_phde_absdev,EDA_phde_kurtosis,EDA_phde_skewness,EDA_tonic_mean,EDA_tonic_median,EDA_tonic_max,EDA_tonic_min,EDA_tonic_vrange,EDA_tonic_var,EDA_tonic_stddev,EDA_tonic_avgder,EDA_tonic_maxgra,EDA_tonic_absdev,EDA_tonic_kurtosis,EDA_tonic_skewness,EDA_one_nbpks,Resp_raw_mean,Resp_raw_median,Resp_raw_max,Resp_raw_min,Resp_raw_vrange,Resp_raw_var,Resp_raw_stddev,Resp_raw_avgder,Resp_raw_maxgra,Resp_raw_absdev,Resp_raw_kurtosis,Resp_raw_skewness,Resp_fre_b0,Resp_fre_b1,Resp_fre_b2,Resp_fre_b3,Resp_fre_b4,Resp_RR_mean,Resp_RR_median,Resp_RR_max,Resp_RR_min,Resp_RR_vrange,Resp_RR_var,Resp_RR_stddev,Resp_RR_avgder,Resp_RR_maxgra,Resp_RR_absdev,Resp_RR_kurtosis,Resp_RR_skewness,Resp_one_Num,Resp_Ampup_mean,Resp_Ampup_median,Resp_Ampup_max,Resp_Ampup_min,Resp_Ampup_vrange,Resp_Ampup_var,Resp_Ampup_stddev,Resp_Ampup_avgder,Resp_Ampup_maxgra,Resp_Ampup_absdev,Resp_Ampup_kurtosis,Resp_Ampup_skewness,sujet,eventidx
0,-0.000433,-0.001563,1.362941,-0.431258,1.794200,0.052222,0.228521,7.570737e-07,0.091472,989.760063,16.354104,3.389563,6.933086e-07,4.172811e-07,4.704904e-07,0.672599,0.671933,0.697930,0.642936,0.054994,0.000295,0.017168,-0.000727,0.042996,0.163984,-0.852771,-0.067526,89.093609,89.244904,90.859910,87.083095,3.776815,1.475273,1.214608,0.099978,2.450904,13.578212,-1.386125,-0.126059,0.888483,2.217476,0.223986,1.687539e-16,-0.233595,2.688783,-2.478834,5.167617,1,1,0.004734,0.205996,304.173761,0.398459,0.831165,0.2600,-1.243450e-16,0.019232,2.484002,-3.541387,6.025389,1,1,0.009347,0.133129,265.627831,2.322822,-0.190190,0.1875,10.772065,10.790836,11.223389,10.569916,0.653472,0.020993,0.144890,0.001607,0.016579,44.959358,0.566925,0.498010,0.210102,0.448681,0.217691,0.093320,0.080717,0.176540,0.194834,0.627387,-0.025151,0.652538,0.020852,0.144401,0.001602,0.016579,44.783395,0.586327,0.506490,0.001602,0.000178,0.016579,-0.003256,0.019834,0.000016,0.003942,-0.000004,0.001202,0.980919,4.801990,2.112266,10.595524,10.595831,10.596002,10.593851,0.002151,3.657302e-07,0.000605,0.000005,1.760455e-05,0.177378,0.301226,-1.228996,3,2.664535e-17,0.003874,0.692203,-0.704760,1.396963,0.184940,0.430046,0.000088,0.044870,153.459279,-1.400396,0.009701,0.046027,0.001196,0.001073,0.178765,0.003074,0.404393,0.408163,0.416667,0.388350,0.028317,0.000211,0.014530,0.004252,0.028317,0.028317,-1.5,-1.089000,4,0.611574,0.647131,0.692203,0.459832,0.232371,0.010692,0.103402,0.077457,0.190491,0.238754,3.289614,-1.735312,2,18
0,-0.000110,-0.002703,1.348619,-0.417872,1.766491,0.056090,0.236834,

In [151]:
phyEventdf.sort(['eventidx'],inplace=True)

In [152]:
phyEventdf.to_csv('./../out/feature/phyBeforeEvent14s.csv',encoding='utf-16',index=False)

In [75]:
phyEventdf=pd.read_csv('./../out/feature/phyEvent.csv',encoding='utf-16',index_col='eventidx')
phyEventdf.head()

ValueError: Index eventidx invalid

## Normalize within groupby(sujet) group

In [407]:
featureTrans=phyEventdf.groupby('sujet').transform(lambda x: (x - x.mean()) / x.std())
# Concat sequence info with normalized features
phyEventNormdf=pd.concat([phyEventdf['sujet'], featureTrans], axis=1)
phyEventNormdf.to_csv('./../out/feature/phyEventNorm.csv',encoding='utf-16')

phyEventNormdf.head()

,sujet,ECG_mean,ECG_median,ECG_max,ECG_min,ECG_vrange,ECG_var,ECG_std_dev,ECG_avg_der,ECG_max_gra,ECG_abs_dev,ECG_kurtosis,ECG_skewness,ECG_b0,ECG_b1,ECG_b2,IBI_mean,IBI_median,IBI_max,IBI_min,IBI_vrange,IBI_var,IBI_std_dev,IBI_avg_der,IBI_max_gra,IBI_abs_dev,IBI_kurtosis,IBI_skewness,HR_mean,HR_median,HR_max,HR_min,HR_vrange,HR_var,HR_std_dev,HR_avg_der,HR_max_gra,HR_abs_dev,HR_kurtosis,HR_skewness,HR_b0,HR_b1,HR_b2,EMG_mean_f,EMG_median_f,EMG_max_f,EMG_min_f,EMG_vrange_f,EMG_var_f,EMG_std_dev_f,EMG_avg_der_f,EMG_max_gra_f,EMG_abs_dev_f,EMG_kurtosis_f,EMG_skewness_f,emg_ct_f,EMG_mean_f_s,EMG_median_f_s,EMG_max_f_s,EMG_min_f_s,EMG_vrange_f_s,EMG_var_f_s,EMG_std_dev_f_s,EMG_avg_der_f_s,EMG_max_gra_f_s,EMG_abs_dev_f_s,EMG_kurtosis_f_s,EMG_skewness_f_s,emg_ct_f_s,EDA_mean,EDA_median,EDA_max,EDA_min,EDA_vrange,EDA_var,EDA_std_dev,EDA_avg_der,EDA_max_gra,EDA_abs_dev,EDA_kurtosis,EDA_skewness,EDA_b0,EDA_b1,EDA_b2,EDA_b3,EDA_b4,EDA_ph_mean,EDA_ph_median,EDA_ph_max,EDA_ph_min,EDA_ph_vrange,EDA_ph_var,EDA_ph_std_dev,EDA_ph_avg_der,EDA_ph_max_gra,EDA_ph_abs_dev,EDA_ph_kurtosis,EDA_ph_skewness,EDA_ph_de_mean,EDA_ph_de_median,EDA_ph_de_max,EDA_ph_de_min,EDA_ph_de_vrange,EDA_ph_de_var,EDA_ph_de_std_dev,EDA_ph_de_avg_der,EDA_ph_de_max_gra,EDA_ph_de_abs_dev,EDA_ph_de_kurtosis,EDA_ph_de_skewness,EDA_to_mean,EDA_to_median,EDA_to_max,EDA_to_min,EDA_to_vrange,EDA_to_var,EDA_to_std_dev,EDA_to_avg_der,EDA_to_max_gra,EDA_to_abs_dev,EDA_to_kurtosis,EDA_to_skewness,EDA_nbpks,rsp_mean,rsp_median,rsp_max,rsp_min,rsp_vrange,rsp_var,rsp_std_dev,rsp_avg_der,rsp_max_gra,rsp_abs_dev,rsp_kurtosis,rsp_skewness,Resp_b0,Resp_b1,Resp_b2,Resp_b3,Resp_b4,RR_mean,RR_median,RR_max,RR_min,RR_vrange,RR_var,RR_std_dev,RR_avg_der,RR_max_gra,RR_abs_dev,RR_kurtosis,RR_skewness,ResNum,ResAmpup_mean,ResAmpup_median,ResAmpup_max,ResAmpup_min,ResAmpup_vrange,ResAmpup_var,ResAmpup_std_dev,ResAmpup_avg_der,ResAmpup_max_gra,ResAmpup_abs_dev,ResAmpup_kurtosis,ResAmpup_skewness
eventidx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18,2,-0.676255,0.017119,-1.789465,0.489156,-1.369097,-1.972292,-1.992517,0.646479,-2.047862,-2.032584,0.119788,-0.023641,-0.140529,-0.129799,-0.144393,-0.577455,-0.561742,-0.910747,-0.489491,-0.834886,-0.814594,-0.896116,-0.093437,-0.056815,-0.704704,-0.592195,-0.402395,0.523654,0.482722,0.320997,0.823054,-0.698447,-0.833899,-0.892813,0.133584,-0.204342,-0.929016,1.023358,1.115465,-0.832009,-0.716861,-0.828606,1.949167,-0.284954,-0.168653,0.203572,-0.295810,NaN,NaN,-0.097186,-0.219086,0.717267,-0.530593,-0.546654,0.541704,1.057728,-0.164334,-0.151502,-0.306138,0.139363,NaN,NaN,1.025327,-0.929814,-0.701643,-0.228629,-0.053092,-0.657976,1.021038,1.034712,1.089670,1.055714,0.778201,0.070276,0.429115,-0.356717,1.332357,0.272205,1.630599,1.165857,0.723837,0.105815,0.260943,-0.388405,-0.663828,-0.204143,0.042022,0.652698,-1.087864,0.949249,0.145629,0.518975,-0.723502,1.344974,0.343950,1.433594,0.696342,-0.723502,-0.902155,1.344974,-1.434768,1.462742,1.145663,1.312763,-0.075132,0.051015,1.167984,1.420754,1.009739,1.052667,1.073759,1.003822,1.025521,0.437138,0.012632,0.437045,0.778770,0.858858,0.434437,0.229890,-0.874721,-0.406760,0.034727,0.275370,-0.799718,0.760197,-0.786798,-0.546275,-0.555342,-0.070817,-0.793732,-0.486058,-0.819339,0.380596,-0.545614,-0.471131,-0.559775,-0.263718,-0.658503,1.029901,0.850293,-0.439644,1.554697,-1.253759,-0.800453,-1.220160,-0.055987,-0.981529,-1.063106,-1.180521,0.294533,0.491614,-0.449223,-0.368783,-0.754732,-0.103999,-0.938180,-0.551174,-0.927930,-0.123218,-0.731820,-0.953438,2.560772,-2.520112
19,2,-0.094325,-0.983920,-2.077616,0.630765,-1.632249,-1.873250,-1.886956,-0.115413,-2.066838,-1.631223,-1.030940,-0.976781,-0.275926,-0.261269,-0.277212,-1.490723,-1.247877,-1.466031,-2.063415,0.032288,-0.070972,0.061930,-0.015553,-1.470516,0.312200,-0.025866,-1.870082,1.558286,1.233633,2.217061,1.615570,0.841778,1

## Normalize with minus baseline

#### Calculate baseline feature

In [418]:
basefilepath='./../biopacBase20/'
phyBase20df=pd.DataFrame()
for fn in os.listdir(basefilepath): 
    filebase=basefilepath+fn
    sujet=int(fn.split('_')[1])
    chunkdf=pd.read_csv(filebase,sep='\t',nrows=20000,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
    currentfeature=allfeature(chunkdf)
    currentfeature['sujet']=sujet
    phyBase20df=phyBase20df.append(currentfeature)

In [419]:
phyBase20df.to_csv('./../out/feature/phyEventBase.csv',encoding='utf-16')

In [420]:
phyBase20df.sort(['sujet'], inplace=True)

In [421]:
left=phyEventdf[['sujet']]
right=phyBase20df
phybasedup = pd.ordered_merge(left, right, fill_method='ffill')
phybasedup.shape

(1730, 165)

In [422]:
df1=phyEventdf.iloc[:,0:122]
df2=phybasedup.iloc[:,1:123]
phyEventSubdf=pd.DataFrame(df1.values-df2.values, columns=df1.columns, index=df1.index)
phyEventSubdf.to_csv('./../out/feature/phyEventSubdf.csv',encoding='utf-16')
phyEventSubdf.head()

,ECG_mean,ECG_median,ECG_max,ECG_min,ECG_vrange,ECG_var,ECG_std_dev,ECG_avg_der,ECG_max_gra,ECG_abs_dev,ECG_kurtosis,ECG_skewness,ECG_b0,ECG_b1,ECG_b2,IBI_mean,IBI_median,IBI_max,IBI_min,IBI_vrange,IBI_var,IBI_std_dev,IBI_avg_der,IBI_max_gra,IBI_abs_dev,IBI_kurtosis,IBI_skewness,HR_mean,HR_median,HR_max,HR_min,HR_vrange,HR_var,HR_std_dev,HR_avg_der,HR_max_gra,HR_abs_dev,HR_kurtosis,HR_skewness,HR_b0,HR_b1,HR_b2,EMG_mean_f,EMG_median_f,EMG_max_f,EMG_min_f,EMG_vrange_f,EMG_var_f,EMG_std_dev_f,EMG_avg_der_f,EMG_max_gra_f,EMG_abs_dev_f,EMG_kurtosis_f,EMG_skewness_f,emg_ct_f,EMG_mean_f_s,EMG_median_f_s,EMG_max_f_s,EMG_min_f_s,EMG_vrange_f_s,EMG_var_f_s,EMG_std_dev_f_s,EMG_avg_der_f_s,EMG_max_gra_f_s,EMG_abs_dev_f_s,EMG_kurtosis_f_s,EMG_skewness_f_s,emg_ct_f_s,EDA_mean,EDA_median,EDA_max,EDA_min,EDA_vrange,EDA_var,EDA_std_dev,EDA_avg_der,EDA_max_gra,EDA_abs_dev,EDA_kurtosis,EDA_skewness,EDA_b0,EDA_b1,EDA_b2,EDA_b3,EDA_b4,EDA_ph_mean,EDA_ph_median,EDA_ph_max,EDA_ph_min,EDA_ph_vrange,EDA_ph_var,EDA_ph_std_dev,EDA_ph_avg_der,EDA_ph_max_gra,EDA_ph_abs_dev,EDA_ph_kurtosis,EDA_ph_skewness,EDA_ph_de_mean,EDA_ph_de_median,EDA_ph_de_max,EDA_ph_de_min,EDA_ph_de_vrange,EDA_ph_de_var,EDA_ph_de_std_dev,EDA_ph_de_avg_der,EDA_ph_de_max_gra,EDA_ph_de_abs_dev,EDA_ph_de_kurtosis,EDA_ph_de_skewness,EDA_to_mean,EDA_to_median,EDA_to_max,EDA_to_min,EDA_to_vrange,EDA_to_var,EDA_to_std_dev,EDA_to_avg_der,EDA_to_max_gra,EDA_to_abs_dev,EDA_to_kurtosis,EDA_to_skewness,EDA_nbpks
eventidx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18,-0.000583,0.001734,-0.037641,-0.004439,-0.033202,-0.009595,-0.019916,0.000001,-0.004764,-265.365879,2.364865,0.194687,-0.000000,1.583804e-08,-0.000000,0.066863,0.070496,0.056997,0.053997,0.003000,-0.000151,-0.003471,-0.002123,9.999500e-03,-0.114994,0.117688,-0.615224,-10.193700,-11.026558,-10.443859,-6.693165,-3.750694,-8.763879,-1.618407,0.345585,1.164651,-47.513263,0.944971,1.150878,-2.417298,-7.771146,-0.275758,2.220446e-16,-0.152048,-0.122946,1.183488,-1.306434,3.330669e-16,2.220446e-16,0.003567,0.024716,34.616758,-0.802039,0.204479,0.04875,6.838974e-16,0.027143,-0.306263,-0.420757,0.114494,3.330669e-16,2.220446e-16,0.000321,-0.085963,-44.871932,0.498157,-0.384625,-0.03250,1.239777,1.327913,1.354812,1.107049,0.247762,0.014053,0.025925,-0.000981,0.006537,4.432095,0.592688,0.062604,0.002243,0.048427,-0.034215,-0.002034,-0.003208,-0.122577,-0.010683,-0.031124,-0.290934,0.259810,0.011060,0.020634,-0.001155,0.006386,-0.670479,0.533154,-0.110700,-0.001155,-0.000660,0.006386,-0.004387,0.010773,0.000009,0.001201,-0.000000,-0.000119,0.721257,2.271259,0.084630,1.362354,1.382005,1.391795,1.263862,0.127934,0.001627,0.039016,0.000174,0.000383,24.268283,-2.274870,-2.272968,-2
19,-0.000339,0.001125,-0.059749,0.008300,-0.068049,-0.009071,-0.018785,0.000001,-0.004847,-223.534694,1.429241,0.119269,-0.000001,-2.703419e-07,-0.000001,0.035256,0.046498,0.033998,0.007000,0.026999,0.000208,0.004080,-0.001967,-7.999600e-03,0.040998,0.452340,-1.229324,-5.679642,-7.577320,-1.839824,-3.101813,1.261989,2.568867,0.348012,0.304792,1.661853,-0.310595,0.710351,1.542552,0.539757,1.834795,0.021742,-1.509903e-16,-0.136338,-0.676119,0.726169,-1.402289,3.330669e-16,2.220446e-16,0.002281,0.016389,27.554605,-0.822509,-0.173123,0.10125,1.776357e-16,0.104052,0.159341,0.089700,0.069641,3.330669e-16,2.220446e-16,-0.002132,-0.091216,-29.397028,0.746698,-0.033613,-0.07500,0.454928,0.561865,-0.009197,0.572254,-0.581451,-0.059421,-0.173335,-0.000813,-0.005451,-107.306479,-1.119877,-0.739992,0.002231,0.038648,-0.022182,-0.004252,-0.003292,-0.185293,-0.076731,-0.651520,-0.066077,-0.585443,-0.059541,-0.174863,-0.000805,-0.005425,-108.270988,-1.108519,-0.714424,-0.000805,-0.000008,-0.005425,0.001714,-0.007138,-0.000008,-0.001689,0.000001,-0.000303,-0.721554,-1.178947,-1.038254,0.640221,0.638766,0.643218,0.638581,0.004637,0.000010,0.002146,-0.000009,-0.000009,1.504297,-2.753815,-0.490310,3
20,-0.000269

In [423]:
phyEventSubNormdf=pd.concat([phyEventdf['sujet'], phyEventSubdf], axis=1)
phyEventSubNormTrans=phyEventSubNormdf.groupby('sujet').transform(lambda x: (x - x.mean()) / x.std())
# Concat sequence info with normalized features
phyEventSubNormdf=pd.concat([phyEventdf['sujet'], phyEventSubNormTrans], axis=1)
phyEventSubNormdf.to_csv('./../out/feature/phyEventSubNorm.csv',encoding='utf-16')

phyEventSubNormdf.head()

,sujet,ECG_mean,ECG_median,ECG_max,ECG_min,ECG_vrange,ECG_var,ECG_std_dev,ECG_avg_der,ECG_max_gra,ECG_abs_dev,ECG_kurtosis,ECG_skewness,ECG_b0,ECG_b1,ECG_b2,IBI_mean,IBI_median,IBI_max,IBI_min,IBI_vrange,IBI_var,IBI_std_dev,IBI_avg_der,IBI_max_gra,IBI_abs_dev,IBI_kurtosis,IBI_skewness,HR_mean,HR_median,HR_max,HR_min,HR_vrange,HR_var,HR_std_dev,HR_avg_der,HR_max_gra,HR_abs_dev,HR_kurtosis,HR_skewness,HR_b0,HR_b1,HR_b2,EMG_mean_f,EMG_median_f,EMG_max_f,EMG_min_f,EMG_vrange_f,EMG_var_f,EMG_std_dev_f,EMG_avg_der_f,EMG_max_gra_f,EMG_abs_dev_f,EMG_kurtosis_f,EMG_skewness_f,emg_ct_f,EMG_mean_f_s,EMG_median_f_s,EMG_max_f_s,EMG_min_f_s,EMG_vrange_f_s,EMG_var_f_s,EMG_std_dev_f_s,EMG_avg_der_f_s,EMG_max_gra_f_s,EMG_abs_dev_f_s,EMG_kurtosis_f_s,EMG_skewness_f_s,emg_ct_f_s,EDA_mean,EDA_median,EDA_max,EDA_min,EDA_vrange,EDA_var,EDA_std_dev,EDA_avg_der,EDA_max_gra,EDA_abs_dev,EDA_kurtosis,EDA_skewness,EDA_b0,EDA_b1,EDA_b2,EDA_b3,EDA_b4,EDA_ph_mean,EDA_ph_median,EDA_ph_max,EDA_ph_min,EDA_ph_vrange,EDA_ph_var,EDA_ph_std_dev,EDA_ph_avg_der,EDA_ph_max_gra,EDA_ph_abs_dev,EDA_ph_kurtosis,EDA_ph_skewness,EDA_ph_de_mean,EDA_ph_de_median,EDA_ph_de_max,EDA_ph_de_min,EDA_ph_de_vrange,EDA_ph_de_var,EDA_ph_de_std_dev,EDA_ph_de_avg_der,EDA_ph_de_max_gra,EDA_ph_de_abs_dev,EDA_ph_de_kurtosis,EDA_ph_de_skewness,EDA_to_mean,EDA_to_median,EDA_to_max,EDA_to_min,EDA_to_vrange,EDA_to_var,EDA_to_std_dev,EDA_to_avg_der,EDA_to_max_gra,EDA_to_abs_dev,EDA_to_kurtosis,EDA_to_skewness,EDA_nbpks
eventidx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18,2,-0.676255,0.017119,-1.789465,0.489156,-1.369097,-1.972292,-1.992517,0.646479,-2.047862,-2.032584,0.119788,-0.023641,-0.140529,-0.129799,-0.144393,-0.577455,-0.561742,-0.910747,-0.489491,-0.834886,-0.814594,-0.896116,-0.093437,-0.056815,-0.704704,-0.592195,-0.402395,0.523654,0.482722,0.320997,0.823054,-0.698447,-0.833899,-0.892813,0.133584,-0.204342,-0.929016,1.023358,1.115465,-0.832009,-0.716861,-0.828606,1.949167,-0.284954,-0.168653,0.203572,-0.295810,NaN,NaN,-0.097186,-0.219086,0.717267,-0.530593,-0.546654,0.541704,1.057728,-0.164334,-0.151502,-0.306138,0.139363,NaN,NaN,1.025327,-0.929814,-0.701643,-0.228629,-0.053092,-0.657976,1.021038,1.034712,1.089670,1.055714,0.778201,0.070276,0.429115,-0.356717,1.332357,0.272205,1.630599,1.165857,0.723837,0.105815,0.260943,-0.388405,-0.663828,-0.204143,0.042022,0.652698,-1.087864,0.949249,0.145629,0.518975,-0.723502,1.344974,0.343950,1.433594,0.696342,-0.723502,-0.902155,1.344974,-1.434768,1.462742,1.145663,1.312763,-0.075132,0.051015,1.167984,1.420754,1.009739,1.052667,1.073759,1.003822,1.025521,0.437138,0.012632,0.437045,0.778770,0.858858,0.434437,0.229890,-0.874721,-0.406760
19,2,-0.094325,-0.983920,-2.077616,0.630765,-1.632249,-1.873250,-1.886956,-0.115413,-2.066838,-1.631223,-1.030940,-0.976781,-0.275926,-0.261269,-0.277212,-1.490723,-1.247877,-1.466031,-2.063415,0.032288,-0.070972,0.061930,-0.015553,-1.470516,0.312200,-0.025866,-1.870082,1.558286,1.233633,2.217061,1.615570,0.841778,1.010272,1.085349,-0.046046,0.500387,1.214623,0.639362,1.899307,0.935800,1.181447,-0.333013,0.449399,-0.109818,-1.192639,-1.013715,-0.465003,NaN,NaN,-0.840322,-0.456886,0.522238,-0.546595,-1.388257,1.327972,-0.111753,0.607995,0.471556,0.430946,0.076093,NaN,NaN,-0.743345,-1.052035,-0.377927,-0.109578,0.363664,-1.134391,0.030725,0.070215,-0.248042,0.263566,-0.841692,-0.580213,-0.748021,-0.066507,-0.641186,-0.692814,-1.139023,-0.570395,0.718393,-0.135921,0.741966,-0.735024,-0.691545,-0.599154,-0.486118,-0.862897,0.264100,-0.841289,-0.560143,-0.724487,0.036427,-0.636433,-0.662782,-1.267526,-0.512585,0.036427,0.681658,-0.636433,0.664316,-0.687064,-0.600400,-0.630026,0.323490,-0.519548,-0.585012,-1.049035,-0.396853,0.149441,0.142158,0.106266,0.218786,-0.835556,-0.454331,-0.840211,-0.339579,-0.721441,-0.840945,-0.744749,0.979076,0.357949
20,2,0.071147,-0.768972,-2.216446,0.570811,-1.671956,-1.680142,-1.682600,-0.228325,-2.206339

# Process event label

In [443]:
dfAll = pd.read_csv('./../out/Allevent.txt',encoding='utf-16',index_col=0)


,sujet,match,Absolute_sec,event,emotions,arousal,valence
0,1,1,47637,rate,frustration,2,-1
1,1,1,47813,arret du gardien,colere,2,-1
2,1,1,47852,but,colere,2,-3
3,1,1,48157,tir,peur,1,-1
4,1,2,49632,arret du gardien,frustration,2,-2


In [444]:
eventLabel=pd.DataFrame()
eventLabel['eventidx']=dfAll.index
eventLabel['pos']=dfAll['valence'].map(lambda x: 1 if x >0 else 0)
eventLabel['high']=dfAll['arousal'].map(lambda x: 1 if x >1 else 0)
eventLabel.set_index('eventidx',inplace=True)

emolabels, emolevels = pd.factorize(dfAll['emotions'])
eventLabel['emotions']=pd.Series(emolabels)
eventlabels, eventlevels = pd.factorize(dfAll['event'])
eventLabel['event']=pd.Series(eventlabels)
eventLabel.head()

In [489]:
sns.set_palette(sns.color_palette("hls", 8))

In [490]:
def rand_jitter(arr):
    stdev = .02*(max(arr)-min(arr))
    return arr + np.random.randn(len(arr)) * stdev
dfAll['arousalr']=rand_jitter(dfAll['arousal'])
dfAll['valencer']=rand_jitter(dfAll['valence'])

In [494]:
#sns.set_context("notebook", font_scale=1.1)

sns.color_palette("hls", 8)

sns.lmplot('arousalr', 'valencer', 
           data=dfAll, 
           fit_reg=False, 
           hue="emotions",  
           scatter_kws={"marker": "D", 
                        "s": 15})
plt.title('Emotion vs A-V')
plt.xlabel('Arousal')
plt.ylabel('Valence')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.show()

# Learning

In [427]:
phyEventNormdf.columns.values

array([u'sujet', u'ECG_mean', u'ECG_median', u'ECG_max', u'ECG_min',
       u'ECG_vrange', u'ECG_var', u'ECG_std_dev', u'ECG_avg_der',
       u'ECG_max_gra', u'ECG_abs_dev', u'ECG_kurtosis', u'ECG_skewness',
       u'ECG_b0', u'ECG_b1', u'ECG_b2', u'IBI_mean', u'IBI_median',
       u'IBI_max', u'IBI_min', u'IBI_vrange', u'IBI_var', u'IBI_std_dev',
       u'IBI_avg_der', u'IBI_max_gra', u'IBI_abs_dev', u'IBI_kurtosis',
       u'IBI_skewness', u'HR_mean', u'HR_median', u'HR_max', u'HR_min',
       u'HR_vrange', u'HR_var', u'HR_std_dev', u'HR_avg_der',
       u'HR_max_gra', u'HR_abs_dev', u'HR_kurtosis', u'HR_skewness',
       u'HR_b0', u'HR_b1', u'HR_b2', u'EMG_mean_f', u'EMG_median_f',
       u'EMG_max_f', u'EMG_min_f', u'EMG_vrange_f', u'EMG_var_f',
       u'EMG_std_dev_f', u'EMG_avg_der_f', u'EMG_max_gra_f',
       u'EMG_abs_dev_f', u'EMG_kurtosis_f', u'EMG_skewness_f', u'emg_ct_f',
       u'EMG_mean_f_s', u'EMG_median_f_s', u'EMG_max_f_s', u'EMG_min_f_s',
       u'EMG_vrange_f_s', u'

In [428]:
df=phyEventSubNormdf.join(eventLabel)

In [429]:
dfn=df.dropna(axis=1,how='all')

In [430]:
from collections import Counter
Counter(dfn['high'])

Counter({0: 887, 1: 843})

In [433]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn import tree
classdataset=dfn
feature_ls=dfn.columns.values[1:-2]
X=np.array(classdataset[feature_ls])
Y=np.array(classdataset['high']).astype(str)
#clf = RandomForestClassifier(n_estimators=20,max_features=20)
#clf = clf.fit(X, Y)
# clf =tree.DecisionTreeClassifier()
# clf = clf.fit(X, Y)
clf = svm.SVC(kernel='linear', C=1)
#clf = svm.SVR()
clf.fit(X, Y) 
scores = cross_validation.cross_val_score(clf, X, Y, cv=10)
scores
# print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
#               scores.mean(),
#               np.std(scores)))


array([ 0.49425287,  0.58045977,  0.55747126,  0.55491329,  0.52601156,
        0.52601156,  0.49710983,  0.51162791,  0.47093023,  0.52906977])

In [99]:
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.barh(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.yticks(range(X.shape[1]), [feature_ls[i] for i in indices])
plt.ylim([-1, X.shape[1]])
plt.show()

Feature ranking:
1. feature 8 (0.014009)
2. feature 84 (0.013742)
3. feature 48 (0.013530)
4. feature 47 (0.012770)
5. feature 100 (0.012717)
6. feature 65 (0.011716)
7. feature 116 (0.011701)
8. feature 13 (0.011344)
9. feature 53 (0.011183)
10. feature 55 (0.011160)
11. feature 58 (0.010968)
12. feature 42 (0.010735)
13. feature 103 (0.010479)
14. feature 1 (0.010423)
15. feature 49 (0.010401)
16. feature 59 (0.010286)
17. feature 24 (0.010274)
18. feature 38 (0.010271)
19. feature 37 (0.010089)
20. feature 82 (0.010039)
21. feature 15 (0.010032)
22. feature 115 (0.009962)
23. feature 4 (0.009947)
24. feature 92 (0.009938)
25. feature 45 (0.009926)
26. feature 6 (0.009703)
27. feature 81 (0.009681)
28. feature 62 (0.009610)
29. feature 110 (0.009593)
30. feature 25 (0.009583)
31. feature 75 (0.009567)
32. feature 106 (0.009478)
33. feature 32 (0.009474)
34. feature 34 (0.009454)
35. feature 21 (0.009405)
36. feature 112 (0.009390)
37. feature 56 (0.009333)
38. feature 80 (0.009323)
3

In [ ]:
def allfeature(chunkdf,sujet, sequence):
    if sequence=='en1':
        NumMatch=0
        Nummitemp=1
    elif sequence=='en2':
        NumMatch=0
        Nummitemp=2
    elif sequence[0]=='b':
        NumMatch=5
        Nummitemp=int(sequence[-1])
    else:
        NumMatch=int(sequence[1])
        Nummitemp=int(sequence[2])
        
    seq_key=['sujet','sequence','NumMatch' ,'Nummitemp']
    seq_info=(sujet,sequence,NumMatch,Nummitemp)
    ecg_key, ecg_feature=ecgfeature(chunkdf['ecg'])
    emgf_key, emgf_feature=emgfeature(chunkdf['emg_f'])
    emgf_key=[i+'_f' for i in emgf_key]
    emgs_key, emgs_feature=emgfeature(chunkdf['emg_s'])
    emgs_key=[i+'_s' for i in emgf_key]
    eda_key, eda_feature=edafeature(chunkdf['eda'])
    rsp_key, rsp_feature=rspfeature(chunkdf['rsp'])
    
    key = seq_key + ecg_key + emgf_key + emgs_key + eda_key + rsp_key
    feature= seq_info + ecg_feature + emgf_feature + emgs_feature + eda_feature + rsp_feature
    return pd.DataFrame([list(feature)], columns=key)

In [10]:
sujet=43
sequence='base0'
chunkdf=pd.read_csv('./../biopacMatch/S18_base0.txt',sep='\t',nrows=180000,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
tmpdf=allfeature(chunkdf,sujet,sequence)

In [19]:
sujet=34
sequence='1100'
chunkdf=pd.read_csv('/home/wenlu/Documents/XPFIFA/biopacEvent20s/S_34_1100_.txt',sep='\t',header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
#chunkdf
tmpdf=allfeature(chunkdf,sujet,sequence)

In [20]:
tmpdf

,sujet,sequence,NumMatch,Nummitemp,ECG_mean,ECG_median,ECG_max,ECG_min,ECG_vrange,ECG_var,ECG_std_dev,ECG_avg_der,ECG_max_gra,ECG_abs_dev,ECG_kurtosis,ECG_skewness,ECG_b0,ECG_b1,ECG_b2,IBI_mean,IBI_median,IBI_max,IBI_min,IBI_vrange,IBI_var,IBI_std_dev,IBI_avg_der,IBI_max_gra,IBI_abs_dev,IBI_kurtosis,IBI_skewness,HR_mean,HR_median,HR_max,HR_min,HR_vrange,HR_var,HR_std_dev,HR_avg_der,HR_max_gra,HR_abs_dev,HR_kurtosis,HR_skewness,HR_b0,HR_b1,HR_b2,EMG_mean_f,EMG_median_f,EMG_max_f,EMG_min_f,EMG_vrange_f,EMG_var_f,EMG_std_dev_f,EMG_avg_der_f,EMG_max_gra_f,EMG_abs_dev_f,EMG_kurtosis_f,EMG_skewness_f,emg_ct_f,EMG_mean_f_s,EMG_median_f_s,EMG_max_f_s,EMG_min_f_s,EMG_vrange_f_s,EMG_var_f_s,EMG_std_dev_f_s,EMG_avg_der_f_s,EMG_max_gra_f_s,EMG_abs_dev_f_s,EMG_kurtosis_f_s,EMG_skewness_f_s,emg_ct_f_s,EDA_mean,EDA_median,EDA_max,EDA_min,EDA_vrange,EDA_var,EDA_std_dev,EDA_avg_der,EDA_max_gra,EDA_abs_dev,EDA_kurtosis,EDA_skewness,EDA_b0,EDA_b1,EDA_b2,EDA_b3,EDA_b4,EDA_ph_mean,EDA_ph_median,EDA_ph_max,EDA_ph_min,EDA_ph_vrange,EDA_ph_var,EDA_ph_std_dev,EDA_ph_avg_der,EDA_ph_max_gra,EDA_ph_abs_dev,EDA_ph_kurtosis,EDA_ph_skewness,EDA_ph_de_mean,EDA_ph_de_median,EDA_ph_de_max,EDA_ph_de_min,EDA_ph_de_vrange,EDA_ph_de_var,EDA_ph_de_std_dev,EDA_ph_de_avg_der,EDA_ph_de_max_gra,EDA_ph_de_abs_dev,EDA_ph_de_kurtosis,EDA_ph_de_skewness,EDA_to_mean,EDA_to_median,EDA_to_max,EDA_to_min,EDA_to_vrange,EDA_to_var,EDA_to_std_dev,EDA_to_avg_der,EDA_to_max_gra,EDA_to_abs_dev,EDA_to_kurtosis,EDA_to_skewness,EDA_nbpks,rsp_mean,rsp_median,rsp_max,rsp_min,rsp_vrange,rsp_var,rsp_std_dev,rsp_avg_der,rsp_max_gra,rsp_abs_dev,rsp_kurtosis,rsp_skewness,Resp_b0,Resp_b1,Resp_b2,Resp_b3,Resp_b4,RR_mean,RR_median,RR_max,RR_min,RR_vrange,RR_var,RR_std_dev,RR_avg_der,RR_max_gra,RR_abs_dev,RR_kurtosis,RR_skewness,ResInter_mean,ResInter_median,ResInter_max,ResInter_min,ResInter_vrange,ResInter_var,ResInter_std_dev,ResInter_avg_der,ResInter_max_gra,ResInter_abs_dev,ResInter_kurtosis,ResInter_skewness,ResAmp_mean,ResAmp_median,ResAmp_max,ResAmp_min,ResAmp_vrange,ResAmp_var,ResAmp_std_dev,ResAmp_avg_der,ResAmp_max_gra,ResAmp_abs_dev,ResAmp_kurtosis,ResAmp_skewness
0,34,1100,1,0,0.000017,0.000069,3.120326,-3.655634,6.77596,0.112981,0.336126,1.939372e-09,0.256003,1792.811663,38.661899,-2.526445,0,3.378934e-08,0,0.891605,0.751462,4.162792,0.375981,3.786811,0.607804,0.779618,0.014894,3.414829,4.888756,18.877192,4.282256,91.616999,81.674783,147.978219,73.20175,74.776469,449.64653,21.20487,-2.70288,18.331503,317.301399,0.960738,1.301479,115.267517,278.502572,49.973495,-4.612762e-16,-0.164215,3.318157,-3.194953,6.51311,1,1,-0.002505,0.172221,582.212803,2.058597,0.614542,0.19975,7.096557e-17,-0.162182,3.323662,-2.146279,5.469941,1,1,-0.000515,0.183589,621.733658,0.834765,0.851493,0.237203,6.039972,5.834911,7.930812,5.759531,2.171281,0.239192,0.489072,0.00256,0.024867,184.688111,4.868708,2.45007,0.117182,0.340261,0.095914,0.020191,0.012361,0.19206,-0.012732,2.086724,-0.084778,2.171501,0.240742,0.490655,0.002582,0.024867,184.780035,4.846128,2.445759,0.002582,0.000064,0.024867,-0.003091,0.027958,0.000037,0.006114,-0.000003,0.001813,2.51626,3.776759,2.189708,5.847911,5.845097,5.86196,5.844085,0.017875,0.000026,0.005105,-0.000022,4.807318e-08,2.940715,0.315688,1.258529,1,-0.025235,-0.03874,4.213715,-2.946201,7.159916,1.985146,1.408952,0.000764,0.181199,861.707497,0.885965,0.596066,0.247638,0.20723,0.759143,0.023181,0.000999,0.28608,0.286455,0.302964,0.268446,0.034519,0.000277,0.016645,-0.001891,0.02167,0.056189,-4.564969,-0.058702,3.60625,3.4,4.675,2.95,1.725,0.594323,0.770923,-0.066667,1.025,2.225,0.97153,1.226949,-0.085402,0.459371,7.148304,-6.711798,13.860102,13.990926,3.740445,-0.887713,11.653379,35.037744,0.218205,0.061326


In [ ]:
PhysioMat_feature = pd.DataFrame()
bioMatchPath="./../biopacMatch/"
sampling_rate=1000
for fn in os.listdir(bioMatchPath):
    print fn
    sujet=re.split('_|\.|S', fn)[1]
    sequence=re.split('_|\.|S', fn)[2]
    if sequence[0]=='b':
        nr=180000
    else:
        nr=240000
    matchFilename=bioMatchPath+fn
    print (matchFilename,sujet,sequence)
    chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=nr,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
    print "read ok"
    print len(chunkdf)
    tmpdf=allfeature(chunkdf,sujet,sequence)
    print (matchFilename,sujet,sequence)
    PhysioMat_feature=PhysioMat_feature.append(tmpdf, ignore_index=True)
    

In [2]:
mbf=pd.read_csv('./../out/matbutFeature.csv',encoding='utf-16')
def rellevel(row):
    if (row['niveaumat']-row['niveausub'])<0:
        return 1
    elif (row['niveaumat']-row['niveausub'])==0:
        return 2
    elif (row['niveaumat']-row['niveausub'])>0:
        return 3
mbf['relniveau']=mbf.apply(rellevel,axis=1)
mbf.head()

,sujet,match,mitemps,niveausub,niveaumat,but,relniveau
0,1,1,1,1,2,-0.5,3
1,1,1,2,1,2,-0.5,3
2,1,2,1,1,1,0.0,2
3,1,2,2,1,1,-0.5,2
4,1,3,1,1,2,0.0,3


In [4]:
fredf = pd.DataFrame()
bioMatchPath="./../biopacMatch/"
sampling_rate=1000
for fn in os.listdir(bioMatchPath):
    #print fn
    sujet=int(re.split('_|\.|S', fn)[1])
    sequence=re.split('_|\.|S', fn)[2]
    mitemps=int(sequence[-1])
    if sequence[0]=='m':
        match=int(sequence[1])
        nr=240000
        matchFilename=bioMatchPath+fn
        chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=nr,header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
        classe=int((mbf.ix[(mbf.sujet == sujet)&(mbf.match == match)&(mbf.mitemps == mitemps),:])['relniveau'])
        sig_ecg= chunkdf['ecg']
        ts, filtered, rpeaks, templates_ts, templates, hr_ts,hr = ecg.ecg(signal=sig_ecg, sampling_rate=1000,show=False)
        freqs,power=st.power_spectrum(filtered, sampling_rate,decibel=False)
        tmp=pd.DataFrame({'sujet':[sujet]*6000,'match':[str(match)+str(mitemps)]*6000,'freq':freqs[0:6000],'power':power[0:6000],'relniveau':[classe]*6000})
        fredf=fredf.append(tmp, ignore_index=True)

In [10]:
fredf.to_csv('./../fre_ecg.csv',encoding='utf-16')

In [8]:
fredf=pd.read_csv('./../fredf.csv',encoding='utf-16',usecols=[1,2,3,4,5])

In [9]:
fredf.head()

,freq,match,power,relniveau,sujet
0,0.000000,12,2.715915e-09,3,48
1,0.004167,12,2.166366e-08,3,48
2,0.008333,12,1.446958e-08,3,48
3,0.012500,12,1.425005e-08,3,48
4,0.016667,12,2.045523e-08,3,48


In [4]:
grouped=fredf.groupby(('sujet'))

In [5]:
s2=grouped.get_group(34)
s2.pivot(index='freq', columns='match', values='power').plot()
plt.show()

In [24]:
s2.pivot(index='freq', columns='match', values='power').head()

match,11,12,21,22,31,32
freq,,,,,,
0.000000,4.786347e-15,2.229995e-10,7.257309e-09,6.438993e-10,8.741819e-11,1.114609e-09
0.004167,1.645718e-10,3.390381e-10,6.591445e-09,1.449521e-09,4.702904e-11,4.897050e-10
0.008333,2.506420e-10,8.774637e-10,6.571131e-10,1.427590e-09,3.222689e-11,1.860097e-10
0.012500,1.797358e-10,3.915283e-10,2.893266e-10,1.186022e-10,2.642407e-11,1.358942e-09
0.016667,1.861849e-10,7.184045e-10,5.274537e-09,7.208299e-10,3.622726e-11,9.112541e-10


In [76]:
df.columns

Int64Index([11, 12, 21], dtype='int64')

In [2]:
n=0
fig, axes = plt.subplots(nrows=12, ncols=5)
color_sequence = ['#1f77b4', '#aec7e8', '#ff7f0e']
for name, group in grouped:
#     print group.pivot(index='freq', columns='match', values='power')[[12,22,32]]
    if name !=34:
        indx=(name-1)/5
        indy=(name-1)%5
        df=group.pivot(index='freq', columns='match', values='power')[[11,21,31]]
        with pd.plot_params.use('x_compat', True):
            clr=color_sequence[int((mbf.ix[(mbf.sujet == name)&(mbf.match == 1)&(mbf.mitemps == 1),:])['relniveau'])-1]
            df[11].plot(color=clr,ax=axes[indx,indy]);axes[indx,indy].set_title('S'+str(name))
            clr=color_sequence[int((mbf.ix[(mbf.sujet == name)&(mbf.match == 2)&(mbf.mitemps == 1),:])['relniveau'])-1]
            df[21].plot(color=clr,ax=axes[indx,indy],);axes[indx,indy].set_title('S'+str(name))
            clr=color_sequence[int((mbf.ix[(mbf.sujet == name)&(mbf.match == 3)&(mbf.mitemps == 1),:])['relniveau'])-1]
            df[31].plot(color=clr,ax=axes[indx,indy],);axes[indx,indy].set_title('S'+str(name))
plt.show()

NameError: name 'grouped' is not defined

In [7]:
n=0
fig, axes = plt.subplots(nrows=12, ncols=5)
for name, group in grouped:
#     print group.pivot(index='freq', columns='match', values='power')[[12,22,32]]
        indx=(name-1)/5
        indy=(name-1)%5
        df=group.pivot(index='freq', columns='match', values='power')
        df.plot(ax=axes[indx,indy]);axes[indx,indy].set_title('S'+str(name))
plt.show()

In [84]:
name

34

In [13]:
fredf.pivot(index='sujet',columns='match')[['freq','power','relniveau']]

ValueError: Index contains duplicate entries, cannot reshape